# Benchmarking time series regression models

Time series extrinsic regression, first properly defined in [1] then recently
extended in [2], involves predicting a continuous target variable based on a time
series. It differs from time series forecasting regression in that the target is
not formed from a sliding window, but is some external variable.

This notebook shows you how to use aeon to get benchmarking datasets with aeon and how
 to compare results on these datasets with those published in [2].

## Loading/Downloading data

aeon comes with two regression problems in the datasets module. You can load these
with single problem loaders or the more general load_regression function.

In [3]:
from aeon.datasets import load_cardano_sentiment, load_covid_3month, load_regression

trainX, trainy = load_covid_3month(split="train")
testX, testy = load_regression(split="test", name="Covid3Month")
X, y = load_cardano_sentiment()  # Combines train and test splits
print(trainX.shape, testX.shape, X.shape)

(140, 1, 84) (61, 1, 84) (107, 2, 24)


there are currently 63 problems in the TSER archive hosted on
timeseriesclassification.com. These are listed in the file datasets.tser_data_lists

In [4]:
from aeon.datasets.tser_data_lists import tser_soton

print(sorted(list(tser_soton)))

['AcousticContaminationMadrid', 'AluminiumConcentration', 'AppliancesEnergy', 'AustraliaRainfall', 'BIDMC32HR', 'BIDMC32RR', 'BIDMC32SpO2', 'BarCrawl6min', 'BeijingIntAirportPM25Quality', 'BeijingPM10Quality', 'BeijingPM25Quality', 'BenzeneConcentration', 'BinanceCoinSentiment', 'BitcoinSentiment', 'BoronConcentration', 'CalciumConcentration', 'CardanoSentiment', 'ChilledWaterPredictor', 'CopperConcentration', 'Covid19Andalusia', 'Covid3Month', 'DailyOilGasPrices', 'DailyTemperatureLatitude', 'DhakaHourlyAirQuality', 'ElectricMotorTemperature', 'ElectricityPredictor', 'EthereumSentiment', 'FloodModeling1', 'FloodModeling2', 'FloodModeling3', 'GasSensorArrayAcetone', 'GasSensorArrayEthanol', 'HotwaterPredictor', 'HouseholdPowerConsumption1', 'HouseholdPowerConsumption2', 'IEEEPPG', 'IronConcentration', 'LPGasMonitoringHomeActivity', 'LiveFuelMoistureContent', 'MadridPM10Quality', 'MagnesiumConcentration', 'ManganeseConcentration', 'MethaneMonitoringHomeActivity', 'MetroInterstateTraffic

You can download these datasets directly with aeon load_regression function. By
default it will store the data in a directory called "local_data" in the datasets
module. Set extract pathto specify a different location.


In [5]:
small_problems = [
    "CardanoSentiment",
    "NaturalGasPricesSentiment",
    "Covid3Month",
    "DailyOilGasPrices",
    "BinanceCoinSentiment",
    "Covid19Andalusia",
]

for problem in small_problems:
    X, y = load_regression(name=problem)
    print(problem, X.shape, y.shape)

CardanoSentiment (107, 2, 24) (107,)
NaturalGasPricesSentiment (93, 1, 20) (93,)
Covid3Month (201, 1, 84) (201,)
DailyOilGasPrices (191, 2, 30) (191,)
BinanceCoinSentiment (263, 2, 24) (263,)
Covid19Andalusia (204, 1, 91) (204,)


If you call the function again, it will load from disk rather than downloading again.
 You can specify train/test splits.

In [6]:
for problem in small_problems:
    trainX, trainy = load_regression(name=problem, split="train")
    print(problem, X.shape, y.shape)

CardanoSentiment (204, 1, 91) (204,)
NaturalGasPricesSentiment (204, 1, 91) (204,)
Covid3Month (204, 1, 91) (204,)
DailyOilGasPrices (204, 1, 91) (204,)
BinanceCoinSentiment (204, 1, 91) (204,)
Covid19Andalusia (204, 1, 91) (204,)


## Evaluating a regressor on benchmark data

With the data, it is easy to assess an algorithm performance. We will use the
DummyRegressor as a baseline, and the default scoring



In [18]:
from sklearn.metrics import mean_squared_error

from aeon.regression import DummyRegressor

dummy = DummyRegressor()
performance = []
for problem in small_problems:
    trainX, trainy = load_regression(name=problem, split="train")
    dummy.fit(trainX, trainy)
    testX, testy = load_regression(name=problem, split="test")
    predictions = dummy.predict(testX)
    mse = mean_squared_error(testy, predictions)
    performance.append(mse)
    print(problem, " Dummy score = ", mse)

BinanceCoinSentiment  Dummy score =  0.1317760422312482
CardanoSentiment  Dummy score =  0.09015657223327135
Covid19Andalusia  Dummy score =  0.0009514194090128098
Covid3Month  Dummy score =  0.0019998715745554777
DailyOilGasPrices  Dummy score =  3.243472060265722
NaturalGasPricesSentiment  Dummy score =  0.008141822846139452


## Comparing to published results

How does the dummy compare to the published results in [2]? We can use the method
get_estimator_results to obtain published results.

In [19]:
from aeon.benchmarking import get_available_estimators, get_estimator_results

print(get_available_estimators(task="regression"))
results = get_estimator_results(
    estimators=["DrCIF", "InceptionTime"],
    task="regression",
    datasets=small_problems,
    measure="mse",
)
print(results)

                1NN-DTW
0                1NN-ED
1               5NN-DTW
2                5NN-ED
3                   CNN
4                 DrCIF
5                   FCN
6                  FPCR
7         FPCR-b-spline
8           FreshPRINCE
9               GridSVR
10        InceptionTime
11                RandF
12               ResNet
13                Ridge
14               ROCKET
15                 RotF
16  SingleInceptionTime
17              XGBoost
{'DrCIF': {'BinanceCoinSentiment': 0.1147393141970096, 'CardanoSentiment': 0.0982120290102569, 'Covid19Andalusia': 0.0002131578438176, 'Covid3Month': 0.0018498023495186, 'DailyOilGasPrices': 1.0949503838264991, 'NaturalGasPricesSentiment': 0.0028579077510607}, 'InceptionTime': {'BinanceCoinSentiment': 0.1453576133389811, 'CardanoSentiment': 0.1436599010092788, 'Covid19Andalusia': 0.0002619962000609, 'Covid3Month': 0.0029974128154996, 'DailyOilGasPrices': 2.378411239715724, 'NaturalGasPricesSentiment': 0.0579133105251111}}


this is organised as a dictionary of dictionaries. because we cannot be sure all
results are present for all datasets.

In [20]:
from aeon.benchmarking import get_estimator_results_as_array

results, names = get_estimator_results_as_array(
    estimators=["DrCIF", "InceptionTime"],
    task="regression",
    datasets=small_problems,
    measure="mse",
)
print(results)
print(names)

[[1.14739314e-01 1.45357613e-01]
 [9.82120290e-02 1.43659901e-01]
 [2.13157844e-04 2.61996200e-04]
 [1.84980235e-03 2.99741282e-03]
 [1.09495038e+00 2.37841124e+00]
 [2.85790775e-03 5.79133105e-02]]
['BinanceCoinSentiment', 'CardanoSentiment', 'Covid19Andalusia', 'Covid3Month', 'DailyOilGasPrices', 'NaturalGasPricesSentiment']


we just need to align our results from the website so they are aligned with the
results from our dummy regressor

In [23]:
import numpy as np

paired_sorted = sorted(zip(names, results))
names, _ = zip(*paired_sorted)
sorted_rows = [row for _, row in paired_sorted]
sorted_results = np.array(sorted_rows)
print(names)
print(sorted_results)

('BinanceCoinSentiment', 'CardanoSentiment', 'Covid19Andalusia', 'Covid3Month', 'DailyOilGasPrices', 'NaturalGasPricesSentiment')
[[1.14739314e-01 1.45357613e-01]
 [9.82120290e-02 1.43659901e-01]
 [2.13157844e-04 2.61996200e-04]
 [1.84980235e-03 2.99741282e-03]
 [1.09495038e+00 2.37841124e+00]
 [2.85790775e-03 5.79133105e-02]]


Do the same for our dummy regressor results

In [26]:
paired = sorted(zip(small_problems, performance))
small_problems, performance = zip(*paired)
print(small_problems)
print(performance)
all_results = np.column_stack((sorted_results, performance))
print(all_results)
regressors = ["DrCIF", "InceptionTime", "Dummy"]

('BinanceCoinSentiment', 'CardanoSentiment', 'Covid19Andalusia', 'Covid3Month', 'DailyOilGasPrices', 'NaturalGasPricesSentiment')
(0.1317760422312482, 0.09015657223327135, 0.0009514194090128098, 0.0019998715745554777, 3.243472060265722, 0.008141822846139452)
[[1.14739314e-01 1.45357613e-01 1.31776042e-01]
 [9.82120290e-02 1.43659901e-01 9.01565722e-02]
 [2.13157844e-04 2.61996200e-04 9.51419409e-04]
 [1.84980235e-03 2.99741282e-03 1.99987157e-03]
 [1.09495038e+00 2.37841124e+00 3.24347206e+00]
 [2.85790775e-03 5.79133105e-02 8.14182285e-03]]


## Comparing Regressors

aeon provides visualisation tools to compare regressors. We can plot the results
against each other.

In [ ]:
# from aeon.visualisation import plot_scatter